# Imports

In [58]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from time import time

import warnings

warnings.filterwarnings("ignore")

In [17]:
df = pd.read_csv("../13/data/garments_worker_productivity.csv")
df

,date,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,1/1/2015,Quarter1,sweing,Thursday,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,0.940725
1,1/1/2015,Quarter1,finishing,Thursday,1,0.75,3.94,NaN,960,0,0.0,0,0,8.0,0.886500
2,1/1/2015,Quarter1,sweing,Thursday,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
3,1/1/2015,Quarter1,sweing,Thursday,12,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
4,1/1/2015,Quarter1,sweing,Thursday,6,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0,0.800382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,3/11/2015,Quarter2,finishing,Wednesday,10,0.75,2.90,NaN,960,0,0.0,0,0,8.0,0.628333
1193,3/11/2015,Quarter2,finishing,Wednesday,8,0.70,3.90,NaN,960,0,0.0,0,0,8.0,0.625625
1194,3/11/2015,Quarter2,finishing,Wednesday,7,0.65,3.90,NaN,960,0,0.0,0,0,8.0,0.625625
1195,3/11/2015,Quarter2,finishing,Wednesday,9,0.75,2.90,NaN,1800,0,0.0,0,0,15.0,0.505889


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   date                   1197 non-null   object 
 1   quarter                1197 non-null   object 
 2   department             1197 non-null   object 
 3   day                    1197 non-null   object 
 4   team                   1197 non-null   int64  
 5   targeted_productivity  1197 non-null   float64
 6   smv                    1197 non-null   float64
 7   wip                    691 non-null    float64
 8   over_time              1197 non-null   int64  
 9   incentive              1197 non-null   int64  
 10  idle_time              1197 non-null   float64
 11  idle_men               1197 non-null   int64  
 12  no_of_style_change     1197 non-null   int64  
 13  no_of_workers          1197 non-null   float64
 14  actual_productivity    1197 non-null   float64
dtypes: f

In [19]:
df.dropna(inplace=True)
df

,date,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,1/1/2015,Quarter1,sweing,Thursday,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,0.940725
2,1/1/2015,Quarter1,sweing,Thursday,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
3,1/1/2015,Quarter1,sweing,Thursday,12,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
4,1/1/2015,Quarter1,sweing,Thursday,6,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0,0.800382
5,1/1/2015,Quarter1,sweing,Thursday,7,0.80,25.90,984.0,6720,38,0.0,0,0,56.0,0.800125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187,3/11/2015,Quarter2,sweing,Wednesday,4,0.75,26.82,1054.0,7080,45,0.0,0,0,59.0,0.750051
1188,3/11/2015,Quarter2,sweing,Wednesday,5,0.70,26.82,992.0,6960,30,0.0,0,1,58.0,0.700557
1189,3/11/2015,Quarter2,sweing,Wednesday,8,0.70,30.48,914.0,6840,30,0.0,0,1,57.0,0.700505
1190,3/11/2015,Quarter2,sweing,Wednesday,6,0.70,23.41,1128.0,4560,40,0.0,0,1,38.0,0.700246


Define feature for classification problem
- Have met productivty target yes or no

In [20]:
y = df["actual_productivity"] >= df["targeted_productivity"]

In [21]:
y.value_counts()

True     573
False    118
Name: count, dtype: int64

In [22]:
df_numeric = df.select_dtypes(exclude="object").drop(columns=["actual_productivity"])
df_numeric

,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers
0,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0
2,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5
3,12,0.80,11.41,968.0,3660,50,0.0,0,0,30.5
4,6,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0
5,7,0.80,25.90,984.0,6720,38,0.0,0,0,56.0
...,...,...,...,...,...,...,...,...,...,...
1187,4,0.75,26.82,1054.0,7080,45,0.0,0,0,59.0
1188,5,0.70,26.82,992.0,6960,30,0.0,0,1,58.0
1189,8,0.70,30.48,914.0,6840,30,0.0,0,1,57.0
1190,6,0.70,23.41,1128.0,4560,40,0.0,0,1,38.0


In [23]:
X_train, X_test, y_train, y_test = train_test_split(df_numeric, y, random_state=42)

## Assignment - Fill Out This Table

| model | train score | test score | average fit time |
| ----- | -----   | -------   | ------- |
| KNN | ? | ? | ? |
| Logistic Regression | ? | ? | ? |
| SVC | ? | ? | ? |

In [24]:
knn = KNeighborsClassifier().fit(X_train, y_train)
knn.score(X_test, y_test)

0.838150289017341

In [25]:
lgr = LogisticRegression().fit(X_train, y_train)
lgr.score(X_test, y_test)

0.9017341040462428

In [26]:
svc = SVC().fit(X_train, y_train)
svc.score(X_test, y_test)

0.8439306358381503

In [99]:
def grid_search_meta_function(estimator_name: str, param_grid: dict):
    tic = time()
    estimator = eval(estimator_name)
    extra_args = {"max_iter": 5000} if estimator_name == "SVC" else {}
    display([estimator_name, extra_args])
    grid = GridSearchCV(
        estimator=estimator(**extra_args), param_grid=param_grid, cv=5
    ).fit(X_train, y_train)

    model_aliases = {
        "KNeighborsClassifier": "KNN",
        "LogisticRegression": "Logistic Regression",
        "SVC": "SVC",
    }

    return {
        "model": model_aliases[estimator_name],
        "train score": grid.best_estimator_.score(X_train, y_train),
        "test score": grid.best_estimator_.score(X_test, y_test),
        "average fit time": time() - tic,
    }

In [95]:
grid_search_params = {
    "KNeighborsClassifier": {
        "n_neighbors": range(1, 10),
        "weights": ["uniform", "distance"],
    },
    "LogisticRegression": {
        "penalty": ["l1", "l2"],
        "C": np.logspace(start=-5, stop=5, num=11),
    },
    "SVC": {
        "kernel": ["linear", "rbf"],
    },
}

grid_search_params

{'KNeighborsClassifier': {'n_neighbors': range(1, 10),
  'weights': ['uniform', 'distance']},
 'LogisticRegression': {'penalty': ['l1', 'l2'],
  'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
         1.e+03, 1.e+04, 1.e+05])},
 'SVC': {'kernel': ['linear', 'rbf']}}

In [100]:
metrics_list = [
    grid_search_meta_function(
        estimator_name=estimator_name, param_grid=grid_search_params[estimator_name]
    )
    for estimator_name in list(grid_search_params.keys())
]

results_df = pd.DataFrame(metrics_list)
results_df = results_df.set_index("model")
results_df

['KNeighborsClassifier', {}]

['LogisticRegression', {}]

['SVC', {'max_iter': 5000}]

,train score,test score,average fit time
model,,,
KNN,1.000000,0.838150,0.599050
Logistic Regression,0.876448,0.901734,0.808188
SVC,0.824324,0.843931,0.108322


| model | train score | test score | average fit time |
| ----- | -----   | -------   | ------- |
| KNN | ? | ? | ? |
| Logistic Regression | ? | ? | ? |
| SVC | ? | ? | ? |